# pyiron jobs

In this guide, we discuss how `pyiron` jobs can be created. This is not a discussion on pyiron itself, we assume that you are familiar with pyiron usage and would like to integrate your custom code into `pyiron`. We will use the `simpledescent` code, which is included in the repository. 

There are various levels of integration possible, we will look through each of them.

## Our task

Our task is to integrate the `simpledescent` code. You can read more in `simpledescent/README.md`. First, let us look at what the code can do. We start by importing the minimise method.

In [1]:
from simpledescent.minimise import Minimisation

Now we find the minimum of a 2D Well potential

In [2]:
mini = Minimisation()
mini.minimise([-0.75, 0.25], learning_rate=0.002)

Reached threshold


[-0.3995746868373576, 2e-323]

As we can see there is one necessary input parameter: the initial position. In the above example, it is `[-0.75, 0.25]`. Additionally there is an optional parameter used: `learning_rate`. There are more parameters possible, but we will look at it later.

Our toycode also has a command line interface. 

```
simpledescent --position -0.75 0.25 --lrate 0.002
```

does the same thing as the python code.

## Why create a pyiron job?

Well, we will see..

## Template Job

Template job is the simplest level of integration possible. For this, we will use the command line interface. 

We start with imports

In [3]:
from pyiron_base import TemplateJob, Project

2023-01-26 18:12:19,721 - pyiron_log - WARNING - Configuration file may be read by others - check permissions to secure credential information!


In [4]:
import os

Now lets define a Job class. To start:

In [5]:
class SimpleDescent(TemplateJob):
    def __init__(self, project, job_name):
        super().__init__(project, job_name)
        
        #now we add inputs
        self.input["initial_position"] = [-0.75, 0.25]
        self.input["learning_rate"] = 0.002
        
        #now we add the executable
        self.executable = f'simpledescent --position {self.input["initial_position"][0]} {self.input["initial_position"][1]} > res.out'

The constructor should take `project` and `job_name` as arguments. Afterwards, the input parameters are defined in the form of dictionary. Finally, the executable is defined. In the executable we defined to pipe the out to a file called `res.out`. The `res.out` file should look something like this:

```
Reached threshold
[-0.3995746868373576, 2e-323]
```

In order to store the results within pyiron, we need to parse the output. The file will be created in the working directory of the job. This can be accessed in the job through `self.working_directory`. Then, the output can be collected using the `collect_output` method. Let's implement this:

In [6]:
class SimpleDescent(TemplateJob):
    def __init__(self, project, job_name):
        super().__init__(project, job_name)
        
        #now we add inputs
        self.input["initial_position"] = [-0.75, 0.25]
        self.input["learning_rate"] = 0.002
        
        #now we add the executable
        self.executable = f'simpledescent --position {self.input["initial_position"][0]} {self.input["initial_position"][1]} > res.out'
        
    def collect_output(self):
        file = os.path.join(self.working_directory, "res.out")
        result = []
        with open(file, "r") as fin:
            for line in fin:
                #we split each line and add
                result.append(line.split())
        #the output we want is in line number 1
        self.output["final_position"] = [float(result[1][0]), float(result[1][1])]

In [7]:
pr = Project("s1")

In [8]:
job = pr.create_job(job_type=SimpleDescent, job_name="a1", delete_existing_job=True)

In [9]:
job.run()

The job a1 was saved and received the ID: 34


In [10]:
job.output

DataContainer({'final_position': [-0.3995746868373576, 2e-323]})

Now we need to add 